# MERGE ANALISIS LEXICON DAN MANUAL

In [11]:
import pandas as pd

df = pd.read_excel("Klarifikasi_Kemunculan_Warna_Kuning_Pada_Rangka_Honda.xlsx")

df = df[["id", "description"]]
df.head()

,id,description
0,1,malu sama kaleng sarden dan hongguan yg bisa t...
1,2,"Berguna kagak, membodohi iya.. 💀"
2,3,Berhentikan rangka esaf🤬
3,4,"kuning2 di rangka esaf itu EMAS TAI, bukan KARAT"
4,5,Mana penjelasan honda jngn lepas tangan


In [12]:
import pandas as pd

df_manual = pd.read_excel("labeling manual.xlsx")
df_manual.head()

,text,sentiment_label
0,malu sama kaleng sarden dan hongguan yg bisa t...,Negatif
1,"Berguna kagak, membodohi iya.. 💀",Negatif
2,Berhentikan rangka esaf🤬,Negatif
3,"kuning2 di rangka esaf itu EMAS TAI, bukan KARAT",Negatif
4,Mana penjelasan honda jngn lepas tangan,Negatif


# Cleansing
Menghapus tanda baca,url, spasi dll.

In [13]:
#regular expression (regex)
import re
import string

def cleaning(Text, replacements=None):
    if pd.isnull(Text):  # Check if Text is NaN
        return ""
    # Mengganti huruf yang berulang-ulang ('oooooo' menjadi '00')
    Text = re.sub(r'(.)\1+', r'\1\1', Text)
    # Mengganti 2 atau lebih titik dengan spasi
    Text = re.sub(r'\.{2,}', ' ', Text)
    # Menghapus @username
    Text = re.sub('@[^\s]+','', Text)
    # Menghapus angka
    Text = re.sub('[0-9]+', '', Text)
    # Menghapus URL
    Text = re.sub(r"http\S+", "", Text)
    # Menghapus hashtag
    Text = re.sub(r'#', '', Text)
    # Menghapus spasi, tanda kutip ganda ("), dan tanda kutip tunggal (') dari teks
    Text = Text.strip(' "\'')
    # Mengganti beberapa spasi dengan satu spasi
    Text = re.sub(r'\s+', ' ', Text)
    # Menghapus tanda baca
    Text = Text.translate(str.maketrans("", "", string.punctuation))
    # Menghapus karakter tidak diinginkan menggunakan kamus pengganti khusus jika disediakan
    if replacements:
        for old, new in replacements.items():
            Text = Text.replace(old, new)
    # Mengembalikan teks yang telah dibersihkan
    return Text

In [14]:
df['textClean_cleaning'] = df['description'].apply(cleaning)
df.drop_duplicates(subset=["textClean_cleaning"], keep="first", inplace=True)
df['textClean_cleaning']

0       malu sama kaleng sarden dan hongguan yg bisa t...
1                           Berguna kagak membodohi iya 💀
2                                Berhentikan rangka esaf🤬
3          kuning di rangka esaf itu EMAS TAI bukan KARAT
4                 Mana penjelasan honda jngn lepas tangan
                              ...                        
3410    di bikin las sambungan saja bang pakai Besi PI...
3411                     Kau ini klo ngomong benar kali 😙
3412    Pcx gen bro sma rambut lu aja msih tebel rambu...
3413    Kejadian yang saya rasain pas dulu pasang stik...
3414    Udah sering lihat motot honda beat ahm patah p...
Name: textClean_cleaning, Length: 3344, dtype: object

In [15]:
df_manual['textClean_cleaning_manual'] = df_manual['text'].apply(cleaning)
df_manual['textClean_cleaning_manual']

0       malu sama kaleng sarden dan hongguan yg bisa t...
1                           Berguna kagak membodohi iya 💀
2                                Berhentikan rangka esaf🤬
3          kuning di rangka esaf itu EMAS TAI bukan KARAT
4                 Mana penjelasan honda jngn lepas tangan
                              ...                        
4505    Pcx gen bro sma rambut lu aja msih tebel rambu...
4506    Kejadian yang saya rasain pas dulu pasang stik...
4507    Hee udah lama om Orangnya aja kelewat tutup ma...
4508    MAU UNTUNG AJA LU MANA TANGGUNG JAWABMU LU KIR...
4509    Udah sering lihat motot honda beat ahm patah p...
Name: textClean_cleaning_manual, Length: 4510, dtype: object

# Case Folding, Clean Emoji
Merubah huruf kapital menjadi huruf kecil dan membersihkan emoticon

In [16]:
#regular expression (regex)
import re

def casefolding(text):
  # Mengubah teks ke huruf kecil (lowercase)
  text = text.lower()
  # Menghapus emoticon
  # Pola regex untuk mendeteksi berbagai karakter emoticon dan simbol
  emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
  # Mengganti semua emoticon dan simbol yang terdeteksi dengan string kosong
  text = emoji_pattern.sub(r'', text)
  # Menghapus karakter non-ASCII
  encoded_string = text.encode("ascii", "ignore")
  # Mengubah kembali byte string menjadi string normal
  text = encoded_string.decode()
  # Mengembalikan teks yang sudah dibersihkan
  return text

In [17]:
df['textClean_casefolding'] = df['textClean_cleaning'].apply(casefolding)
df['textClean_casefolding']

0       malu sama kaleng sarden dan hongguan yg bisa t...
1                            berguna kagak membodohi iya 
2                                 berhentikan rangka esaf
3          kuning di rangka esaf itu emas tai bukan karat
4                 mana penjelasan honda jngn lepas tangan
                              ...                        
3410    di bikin las sambungan saja bang pakai besi pi...
3411                      kau ini klo ngomong benar kali 
3412    pcx gen bro sma rambut lu aja msih tebel rambut  
3413    kejadian yang saya rasain pas dulu pasang stik...
3414    udah sering lihat motot honda beat ahm patah p...
Name: textClean_casefolding, Length: 3344, dtype: object

In [18]:
df_manual['textClean_casefolding_manual'] = df_manual['textClean_cleaning_manual'].apply(casefolding)
df_manual['textClean_casefolding_manual']

0       malu sama kaleng sarden dan hongguan yg bisa t...
1                            berguna kagak membodohi iya 
2                                 berhentikan rangka esaf
3          kuning di rangka esaf itu emas tai bukan karat
4                 mana penjelasan honda jngn lepas tangan
                              ...                        
4505    pcx gen bro sma rambut lu aja msih tebel rambut  
4506    kejadian yang saya rasain pas dulu pasang stik...
4507    hee udah lama om orangnya aja kelewat tutup ma...
4508    mau untung aja lu mana tanggung jawabmu lu kir...
4509    udah sering lihat motot honda beat ahm patah p...
Name: textClean_casefolding_manual, Length: 4510, dtype: object

# Stemming
kata yang memiliki imbuhan menjadi kata dasar

In [19]:
#Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [20]:
df['textClean_stemmer'] = df['textClean_casefolding'].apply(stemmer.stem)
df['textClean_stemmer']

0       malu sama kaleng sarden dan hongguan yg bisa t...
1                                    guna kagak bodoh iya
2                                       henti rangka esaf
3          kuning di rangka esaf itu emas tai bukan karat
4                      mana jelas honda jngn lepas tangan
                              ...                        
3410    di bikin las sambung saja bang pakai besi pipa...
3411                       kau ini klo ngomong benar kali
3412      pcx gen bro sma rambut lu aja msih tebel rambut
3413    jadi yang saya rasain pas dulu pasang stiker o...
3414    udah sering lihat motot honda beat ahm patah p...
Name: textClean_stemmer, Length: 3344, dtype: object

In [21]:
df_manual['textClean_stemmer_manual'] = df_manual['textClean_casefolding_manual'].apply(stemmer.stem)
df_manual['textClean_stemmer_manual']

0       malu sama kaleng sarden dan hongguan yg bisa t...
1                                    guna kagak bodoh iya
2                                       henti rangka esaf
3          kuning di rangka esaf itu emas tai bukan karat
4                      mana jelas honda jngn lepas tangan
                              ...                        
4505      pcx gen bro sma rambut lu aja msih tebel rambut
4506    jadi yang saya rasain pas dulu pasang stiker o...
4507    hee udah lama om orang aja lewat tutup mata sa...
4508    mau untung aja lu mana tanggung jawab lu kira ...
4509    udah sering lihat motot honda beat ahm patah p...
Name: textClean_stemmer_manual, Length: 4510, dtype: object

# Slangwords
mengindentifikasi kata-kata slang (kata gaul) lalu akan diganti ke kata yang lebih baku atau umum

In [22]:
# Membaca file CSV 'kbba.txt' yang berisi kamus kata slang dan formalnya
kbba_dictionary = pd.read_csv('kbba.txt', delimiter='\t', names=['slang', 'formal'], header=None, encoding='utf-8')
# Membuat kamus dari DataFrame 'kbba_dictionary', di mana 'slang' adalah key dan 'formal' adalah value
slang_dict = dict(zip(kbba_dictionary['slang'], kbba_dictionary['formal']))
# Menampilkan DataFrame 'kbba_dictionary' untuk melihat isi kamus slang dan formal
kbba_dictionary

def convert_slangword(text):
    words = text.split()
    normalized_words = [slang_dict[word] if word in slang_dict else word for word in words]
    normalized_text = ' '.join(normalized_words)
    return normalized_text

In [23]:
df['textClean_slang'] = df['textClean_stemmer'].apply(convert_slangword)
df['textClean_slang']

0       malu sama kaleng sarden dan kongguan yang bisa...
1                                 berguna tidak bodoh iya
2                                       henti rangka esaf
3       kuning di rangka esaf itu emas kotoran bukan k...
4                    mana jelas honda jangan lepas tangan
                              ...                        
3410    di bikin las sambung saja bang pakai besi pipa...
3411                      kamu ini kalo bicara benar kali
3412    pcx gen saudara laki-laki sama rambut kamu saj...
3413    jadi yang saya rasain pas dulu pasang stiker o...
3414    sudah sering lihat motot honda beat ahm patah ...
Name: textClean_slang, Length: 3344, dtype: object

In [24]:
df_manual['textClean_slang_manual'] = df_manual['textClean_stemmer_manual'].apply(convert_slangword)
df_manual['textClean_slang_manual']

0       malu sama kaleng sarden dan kongguan yang bisa...
1                                 berguna tidak bodoh iya
2                                       henti rangka esaf
3       kuning di rangka esaf itu emas kotoran bukan k...
4                    mana jelas honda jangan lepas tangan
                              ...                        
4505    pcx gen saudara laki-laki sama rambut kamu saj...
4506    jadi yang saya rasain pas dulu pasang stiker o...
4507    hee sudah lama om orang saja lewat tutup mata ...
4508    mau untung saja kamu mana tanggung jawab kamu ...
4509    sudah sering lihat motot honda beat ahm patah ...
Name: textClean_slang_manual, Length: 4510, dtype: object

# Stopword
ini akan menghapus seluruh kata yang dianggap tidak penting, yang mana tidak akan mempengaruhi sentimen pada kalimat. Kata yang tidak penting disini adalah kata sambung seperti, di, ke, ini, dan, dari, segala, itu.

In [25]:
# Import modul StopWord dari library nlp_id untuk penghapusan stopwords
from nlp_id.stopword import StopWord
# Inisialisasi objek StopWord untuk pengolahan stopwords bahasa Indonesia
stopword = StopWord()
# Dapatkan daftar stopwords dari modul StopWord
stopwords = stopword.get_stopword()

# Hapus kata "tidak" dari daftar stopwords
if "tidak" in stopwords:
    stopwords.remove("tidak")
# Fungsi untuk menghapus stopwords dengan pengecualian kata "tidak"
def remove_stopwords_with_exception(text):
    words = text.split()
    cleaned_words = [word for word in words if word not in stopwords]
    return " ".join(cleaned_words).strip()

In [26]:
# Gunakan fungsi remove_stopwords_with_exception pada kolom "textClean_slang"
df["Text_Clean"] = df["textClean_slang"].apply(remove_stopwords_with_exception)
df = df[df['Text_Clean'].str.strip().astype(bool)]
df.dropna(axis=1, how='all', inplace=True)
df["Text_Clean"]

0                    malu kaleng sarden kongguan tahan ah
1                                     berguna tidak bodoh
2                                       henti rangka esaf
3                   kuning rangka esaf emas kotoran karat
4                                      honda lepas tangan
                              ...                        
3410    bikin las sambung bang pakai besi pipa galvani...
3411                                          kalo bicara
3412    pcx gen saudara laki-laki rambut msih tebel ra...
3413    rasain pasang stiker oracal honda cbr dealer c...
3414          lihat motot honda beat ahm patah porek depa
Name: Text_Clean, Length: 3308, dtype: object

In [27]:
# Gunakan fungsi remove_stopwords_with_exception pada kolom "textClean_slang"
df_manual["Text_Clean_manual"] = df_manual["textClean_slang_manual"].apply(remove_stopwords_with_exception)
df_manual["Text_Clean_manual"]

0                    malu kaleng sarden kongguan tahan ah
1                                     berguna tidak bodoh
2                                       henti rangka esaf
3                   kuning rangka esaf emas kotoran karat
4                                      honda lepas tangan
                              ...                        
4505    pcx gen saudara laki-laki rambut msih tebel ra...
4506    rasain pasang stiker oracal honda cbr dealer c...
4507                         hee om orang tutup mata merk
4508    untung tanggung bayar daun mangga kualitas jag...
4509          lihat motot honda beat ahm patah porek depa
Name: Text_Clean_manual, Length: 4510, dtype: object

# Unwanted Word Removal
menghapus Kata-kata yang tidak diinginkan yang akan dihapus dari teks

In [28]:
# Import NLTK untuk tokenisasi
import nltk
from nltk import word_tokenize, sent_tokenize
#Unduh dataset 'punkt' dari NLTK untuk tokenisasi teks
nltk.download('punkt')

# menghapus Kata-kata yang tidak diinginkan yang akan dihapus dari teks
unwanted_words = ['jan','feb','mar','apr','mei','jun','jul','aug','sep','oct','nov','dec','uaddown','weareuad','Iam','https','igshid']

def RemoveUnwantedwords(text):
    word_tokens = word_tokenize(text)
    filtered_sentence = [word for word in word_tokens if not word in unwanted_words]
    return ' '.join(filtered_sentence)

[nltk_data] Downloading package punkt to C:\Users\Yuki Prasetya
[nltk_data]     Mukmin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
df["Text_Clean_lagi"] = df["Text_Clean"].apply(RemoveUnwantedwords)
df["Text_Clean_lagi"]

0                    malu kaleng sarden kongguan tahan ah
1                                     berguna tidak bodoh
2                                       henti rangka esaf
3                   kuning rangka esaf emas kotoran karat
4                                      honda lepas tangan
                              ...                        
3410    bikin las sambung bang pakai besi pipa galvani...
3411                                          kalo bicara
3412    pcx gen saudara laki-laki rambut msih tebel ra...
3413    rasain pasang stiker oracal honda cbr dealer c...
3414          lihat motot honda beat ahm patah porek depa
Name: Text_Clean_lagi, Length: 3308, dtype: object

In [30]:
df_manual["Text_Clean_lagi_manual"] = df_manual["Text_Clean_manual"].apply(RemoveUnwantedwords)
df_manual["Text_Clean_lagi_manual"]

0                    malu kaleng sarden kongguan tahan ah
1                                     berguna tidak bodoh
2                                       henti rangka esaf
3                   kuning rangka esaf emas kotoran karat
4                                      honda lepas tangan
                              ...                        
4505    pcx gen saudara laki-laki rambut msih tebel ra...
4506    rasain pasang stiker oracal honda cbr dealer c...
4507                         hee om orang tutup mata merk
4508    untung tanggung bayar daun mangga kualitas jag...
4509          lihat motot honda beat ahm patah porek depa
Name: Text_Clean_lagi_manual, Length: 4510, dtype: object

# Menghapus kata di bawah 3 huruf

In [31]:
df["Text_Clean_lagi"] = df["Text_Clean_lagi"].str.findall('\w{3,}').str.join(' ')
df["Text_Clean_lagi"]

0                       malu kaleng sarden kongguan tahan
1                                     berguna tidak bodoh
2                                       henti rangka esaf
3                   kuning rangka esaf emas kotoran karat
4                                      honda lepas tangan
                              ...                        
3410    bikin las sambung bang pakai besi pipa galvani...
3411                                          kalo bicara
3412    pcx gen saudara laki laki rambut msih tebel ra...
3413    rasain pasang stiker oracal honda cbr dealer c...
3414          lihat motot honda beat ahm patah porek depa
Name: Text_Clean_lagi, Length: 3308, dtype: object

In [32]:
df_manual["Text_Clean_lagi_manual"] = df_manual["Text_Clean_lagi_manual"].str.findall('\w{3,}').str.join(' ')
df_manual["Text_Clean_lagi_manual"]

0                       malu kaleng sarden kongguan tahan
1                                     berguna tidak bodoh
2                                       henti rangka esaf
3                   kuning rangka esaf emas kotoran karat
4                                      honda lepas tangan
                              ...                        
4505    pcx gen saudara laki laki rambut msih tebel ra...
4506    rasain pasang stiker oracal honda cbr dealer c...
4507                            hee orang tutup mata merk
4508    untung tanggung bayar daun mangga kualitas jag...
4509          lihat motot honda beat ahm patah porek depa
Name: Text_Clean_lagi_manual, Length: 4510, dtype: object

# Tokenizing
proses pemisahan kata, pemisahan kata pada setiap kalimat dilakukan berdasarkan delimeter yaitu adanya spasi pada setiap kata.

In [33]:
# Mengimpor modul NLTK untuk pengolahan bahasa alami
import nltk
# Mengimpor fungsi word_tokenize dari NLTK
from nltk.tokenize import word_tokenize
# Mengunduh dataset 'punkt' dari NLTK yang digunakan untuk tokenisasi kata
nltk.download('punkt')

def tokenize(teks):
    list_teks = []
    words = teks.split(" ")
    i = 0
    while i < len(words):
        if words[i] == "tidak" and i + 1 < len(words) and words[i + 1] == "percaya":
            list_teks.append("tidak percaya")
            i += 2
        else:
            list_teks.append(words[i])
            i += 1
    return list_teks


[nltk_data] Downloading package punkt to C:\Users\Yuki Prasetya
[nltk_data]     Mukmin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
df["Text_Clean_split"] = df["Text_Clean_lagi"].apply(tokenize)
df["Text_Clean_split"].head()

0         [malu, kaleng, sarden, kongguan, tahan]
1                         [berguna, tidak, bodoh]
2                           [henti, rangka, esaf]
3    [kuning, rangka, esaf, emas, kotoran, karat]
4                          [honda, lepas, tangan]
Name: Text_Clean_split, dtype: object

In [35]:
df_manual["Text_Clean_split_manual"] = df_manual["Text_Clean_lagi_manual"].apply(tokenize)
df_manual["Text_Clean_split_manual"].head()

0         [malu, kaleng, sarden, kongguan, tahan]
1                         [berguna, tidak, bodoh]
2                           [henti, rangka, esaf]
3    [kuning, rangka, esaf, emas, kotoran, karat]
4                          [honda, lepas, tangan]
Name: Text_Clean_split_manual, dtype: object

# Export Data

In [36]:
# # Export Data
# from datetime import datetime
# # Get the current date and time
# current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# # Create the filename with the current date and time
# filename = f'hasil_preprocessing_komentar_youtube_{current_time}.xlsx'
# # Export DataFrame to Excel
# df.to_excel(filename, index=False)

In [37]:
# # Export Data
# from datetime import datetime
# # Get the current date and time
# current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# # Create the filename with the current date and time
# filename = f'hasil_labelling_manual.xlsx'
# # Export DataFrame to Excel
# df_manual.to_excel(filename, index=False)

# Labeling Data (lexicon)
proses labeling dengan mencocokkan dan menghitung banyaknya kata positif dan negatif sesuai dengan kamus, lalu data akandihitung jika lebih banyak kata positif dari pada kata negatif maka akan diberikan label positif dan jika lebih banyak kata negatif dari pada kata positif maka akan diberikan label negatif

In [38]:
# Daftar Kata-kata positif bahasa indonesia
df_positive = pd.read_csv('https://raw.githubusercontent.com/YUKIPM24/kamus_lexicon/main/positive.txt', sep='\t',names=['positive'])
list_positive = list(df_positive.iloc[::,0])

# Daftar Kata-Kata negatif bahasa indonesia
df_negative = pd.read_csv('https://raw.githubusercontent.com/YUKIPM24/kamus_lexicon/main/negative.txt', sep='\t',names=['negative'])
list_negative = list(df_negative.iloc[::,0])


In [39]:
# menghitung kata-kata positif/negatif pada dictionary lalu menentukan sentimennya :
def sentiment_kamus_lexicon_id(text):
    score = 0
    positive_words = []
    negative_words = []
    neutral_words = []
    
    for word in text:
        if word in list_positive:
            score += 1
            positive_words.append(word)
        elif word in list_negative:
            score -= 1
            negative_words.append(word)
        else:
            neutral_words.append(word)
    
    polarity = ''
    if score > 0:
        polarity = 'positive'
    elif score < 0:
        polarity = 'negative'
    else:
        polarity = 'neutral'
    
    result = {'positif': positive_words, 'negatif': negative_words, 'neutral': neutral_words}
    return score, polarity, result, positive_words, negative_words, neutral_words


In [40]:
def sentiment_manual_label(text_labels):
    positive_words = []
    negative_words = []
    neutral_words = []
    
    for word, label in text_labels:
        if label.lower() == 'positive':
            positive_words.append(word)
        elif label.lower() == 'negative':
            negative_words.append(word)
        else:
            neutral_words.append(word)
    
    total_positive = len(positive_words)
    total_negative = len(negative_words)
    total_neutral = len(neutral_words)
    
    score = total_positive - total_negative
    
    polarity = ''
    if score > 0:
        polarity = 'positive'
    elif score < 0:
        polarity = 'negative'
    else:
        polarity = 'neutral'
    
    result = {'positif': positive_words, 'negatif': negative_words, 'netral': neutral_words}
    return score, polarity, result, positive_words, negative_words, neutral_words

In [41]:
hasil = df['Text_Clean_split'].apply(sentiment_kamus_lexicon_id)
hasil = list(zip(*hasil))
df['polarity_score'] = hasil[0]
df['polarity'] = hasil[1]
hasil_kata_positive = hasil[3]
hasil_kata_negative = hasil[4]

df = df[df.polarity != 'neutral']
df.dropna(inplace=True)

In [42]:
df_manual['sentiment_label'] = df_manual['sentiment_label'].replace({'Positif': 'positive', 'Negatif': 'negative', 'Netral': 'neutral'})

# Gabungkan Text_Clean_split_manual dengan sentiment_label menjadi satu daftar tuple
df_manual['text_labels'] = df_manual.apply(lambda row: list(zip(row['Text_Clean_split_manual'], row['sentiment_label'].split())), axis=1)

# Terapkan fungsi sentiment analysis ke kolom text_labels
df_manual['sentiment_analysis'] = df_manual['text_labels'].apply(sentiment_manual_label)

# Pisahkan hasil menjadi kolom-kolom terpisah
hasil = list(zip(*df_manual['sentiment_analysis']))
df_manual['polarity_score'] = hasil[0]
df_manual['polarity'] = hasil[1]
df_manual['result'] = hasil[2]
df_manual['positive_words'] = hasil[3]
df_manual['negative_words'] = hasil[4]
df_manual['neutral_words'] = hasil[5]

# Filter DataFrame hanya untuk sentimen yang bukan netral
df_manual = df_manual[df_manual['polarity'] != 'neutral']

## Merge data hasil labeling

In [43]:
len(df), len(df_manual)

(1749, 4510)

In [44]:
data_merge = {
    "text": df["Text_Clean_lagi"],
    "sentiment_label_lexicon": df["polarity"],
    "sentiment_label_manual": df_manual["polarity"]
}

data = pd.DataFrame(data_merge)
data.dropna(inplace=True, axis=0)
data.to_excel("hasil_labeling_lexicon_manual.xlsx",index=False)
data.head()

,text,sentiment_label_lexicon,sentiment_label_manual
0,malu kaleng sarden kongguan tahan,negative,negative
1,berguna tidak bodoh,negative,negative
2,henti rangka esaf,negative,negative
6,karat salah orang produksi salah coba produksi...,negative,negative
7,takut muka dasar honda esaf,negative,negative


In [45]:
# menghitung hasil sentiment analysis
print("jumlah data: ")
print(df['polarity'].value_counts().sum())

print("\ndetail :")
print(df['polarity'].value_counts())

jumlah data: 
1749

detail :
polarity
negative    945
positive    804
Name: count, dtype: int64


In [46]:
# Menghitung jumlah data
print("Jumlah data:")
print(df_manual['polarity'].value_counts().sum())

# Menampilkan detail sentimen
print("\nDetail:")
print(df_manual['polarity'].value_counts())


Jumlah data:
4510

Detail:
polarity
negative    3239
positive    1271
Name: count, dtype: int64
